<a href="https://colab.research.google.com/github/wagner1986/PapyrusTech/blob/main/ManyLocalFeatureMatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kornia
# !pip install kornia_moons --no-deps
#!pip install kornia
#!pip install kornia_moons

In [ ]:
import time
from contextlib import contextmanager
import torch
import numpy as np

def count_correct_matches(matches_matrix):
    n, _ = matches_matrix.shape
    count = 0
    for i in range(n):
        max_match = np.max(matches_matrix[i])
        if matches_matrix[i, i] >= max_match:
            count += 1
    return count,n

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Desativar para garantir a reprodutibilidade

@contextmanager
def medir_tempo(label: str = "Tempo de execução"):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print(f"{label}: {end - start} segundos")

def print_table(matrix):
    num_rows, num_cols = matrix.shape

    # Encontrando o número máximo de dígitos para alinhar corretamente
    max_width = len(str(num_rows * num_cols))

    # Imprimindo os índices das colunas
    print(f"    ".rjust(max_width + 3), end=" |")
    for col_idx in range(num_cols):
        print(f" Col{col_idx} ".center(10), end=" |")
    print()

    # Imprimindo uma linha de separação
    print(f"    ".rjust(max_width + 5)+"-" * (max_width + 1 + 11 * num_cols))

    # Imprimindo as linhas da tabela
    for row_idx in range(num_rows):
        print(f" Row{row_idx}".ljust(max_width + 1), end=" |")  # Adiciona um espaço extra na primeira célula
        for col_idx in range(num_cols):
            print(f" {matrix[row_idx, col_idx]} ".center(10), end=" |")
        print()

# Usando o context manager para medir o tempo de execução
with medir_tempo('Processamento concluído'):
    for _ in range(1000000):
        pass

In [ ]:
import torch.nn as nn
import torchvision
import kornia as K
from abc import ABC, abstractmethod
import torch
from kornia import tensor_to_image


import matplotlib.pyplot as plt

def plot_tensor(tensor):
    """
    Plota um tensor PyTorch como uma imagem.

    Parâmetros:
    - tensor (torch.Tensor): Um tensor 2D para imagens em escala de cinza ou um tensor 3D para imagens RGB.
    """
    if tensor.dim() == 2:
        # Imagem em escala de cinza
        plt.imshow(tensor, cmap='gray')
    elif tensor.dim() == 3:
        # Convertendo o tensor de CxHxW para HxWxC para plotagem
        tensor = tensor.permute(1, 2, 0)
        plt.imshow(tensor)
    else:
        raise ValueError("O tensor deve ser 2D (imagem em escala de cinza) ou 3D (imagem RGB).")

    plt.axis('off')  # Remove os eixos para uma visualização mais limpa
    plt.show()

def plot_image_with_keypoints(image, keypoints):
    """
    Plota uma imagem e seus keypoints.

    Parâmetros:
    - image (torch.Tensor): A imagem original.
    - keypoints (torch.Tensor): Os keypoints detectados.
    """
    # Converter a imagem para o formato do matplotlib (H, W, C) e escala [0, 1] se necessário
    if image.dim() == 3:
        image = image.permute(1, 2, 0)
    if torch.max(image) > 1:
        image = image / 255.0

    plt.imshow(image.cpu().numpy())
    if keypoints.dim() == 3:
        keypoints = keypoints[0]  # Se os keypoints estiverem em um batch, pegue apenas o primeiro

    # Plotar os keypoints
    if keypoints.shape[1] == 2:
        plt.scatter(keypoints[:, 0], keypoints[:, 1], s=20, marker='.', c='r')
    elif keypoints.shape[1] == 3:  # Se os keypoints incluírem a confiança
        plt.scatter(keypoints[:, 0], keypoints[:, 1], s=20 * keypoints[:, 2], marker='.', c='r')

    plt.axis('off')
    plt.show()

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, List, Tuple
from itertools import product
import numpy as np
import torch

class IPreprocessor(ABC):
    """
    Interface para um pré-processador de imagens.
    Define o contrato para classes que implementam operações de pré-processamento em imagens.
    """

    @abstractmethod
    def __call__(self, image: torch.Tensor) -> torch.Tensor:
        """
        Aplica operações de pré-processamento à imagem fornecida.

        Parâmetros:
            image (torch.Tensor): A imagem de entrada como um tensor do PyTorch.

        Retorna:
            torch.Tensor: A imagem após o pré-processamento.
        """
        pass

class IGlobalFeatureStructurer(ABC):
    """
    Interface para a estruturação global de features através de grafo.
    """

    @abstractmethod
    def __call__(self, points: np.ndarray, featuresByPoints: np.ndarray) -> Tuple[np.ndarray,Any]:
        """
        Estrutura features globais de uma imagem em um grafo, utilizando pontos e suas características associadas.

        Args:
            points (np.ndarray): Array de pontos extraídos de uma imagem, onde cada ponto é uma coordenada 2D.
            featuresByPoints (np.ndarray): Array de características associadas a cada ponto.

        Returns:
            Tuple[np.ndarray, Any]: Um grafo representando a estruturação global das features e o objeto Delaunay utilizado para a triangulação.
        """
        pass

class IGlobalMatcher(ABC):
    """
    Interface para a similaridade global de features através de grafo.
    """
    @abstractmethod
    def __call__(self, matrixAdj0: np.ndarray, matrixAdj1: np.ndarray, threshold: float = 0.2) -> int:
        """
        Interface para invocar operações de comparação de grafos.

        Args:
            matrixAdj0: A primeira matriz de adjacência do grafo.
            matrixAdj1: A segunda matriz de adjacência do grafo.
            threshold: O limiar para avaliar correspondências.

        Returns:
            Um valor representando a similaridade entre os dois grafos.
        """
        pass


In [ ]:

import torch
import torchvision
from kornia.feature import LocalFeatureMatcher, GFTTAffNetHardNet, DescriptorMatcher,KeyNetHardNet

# Supondo que 'device' já esteja definido, por exemplo, device = torch.device('cuda' ou 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)
# Carregando a imagem, convertendo para float e normalizando
# image1 = torchvision.io.read_image("image1.jpg").to(torch.float) / 255.0  # Normaliza para [0, 1]
# plot_tensor(image1)

# image1 = image1.to(device)
# print(image1.shape)
# gftt_hardnet_matcher = LocalFeatureMatcher(
#     GFTTAffNetHardNet(25,device=device), DescriptorMatcher('smnn', 0.8)
# )

# input = {"image0": image1[:1][None], "image1": image1[:1][None]}# so funciona em escala de cinza
# out = gftt_hardnet_matcher(input)
# print(type(image1.cpu().data))
# plot_image_with_keypoints(image1.cpu(),out['keypoints0'].detach().cpu())


In [ ]:
import torch
from kornia.feature import LocalFeature, DescriptorMatcher
from typing import Dict, Optional

class PreprocessPipeline(IPreprocessor):
    def __init__(self):
        super(PreprocessPipeline, self).__init__()
        self.transforms = nn.Sequential(
            # Normaliza a imagem para ter valores no intervalo [0, 1]
            K.enhance.Normalize(0.0, 255.0),
            # Redimensiona a imagem para o tamanho desejado
            K.geometry.Resize((200, 200)),
            # Converte a imagem RGB para escala de cinza , necessario comentar para plotar
            K.color.RgbToGrayscale(),
        )

    def __call__(self, image: torch.Tensor) -> torch.Tensor:
        # Normaliza a imagem para ter valores no intervalo [0, 1]
        x = image#.float() / 255.0  # Garante que a operação de divisão é realizada em float
        # Aplica as transformações definidas em __init__
        x = self.transforms(x)
        # Adiciona uma dimensão de batch no início se ainda não houver
        if x.ndim == 3:
            x = x.unsqueeze(0)
        return x

class MyLocalFeatureMatcher:
    def __init__(self, local_feature: LocalFeature, descriptor_matcher: DescriptorMatcher):
        """
        Inicializa o matcher de características locais com um objeto LocalFeature para extração de características
        e um objeto DescriptorMatcher para o matching de descritores.

        Parâmetros:
            local_feature (LocalFeature): Um objeto LocalFeature para extração combinada de detector e descritor.
            descriptor_matcher (DescriptorMatcher): Um objeto DescriptorMatcher configurado para o matching de descritores.
        """
        self.local_feature = local_feature
        self.descriptor_matcher = descriptor_matcher

    def __call__(self, input) -> Dict[str, torch.Tensor]:
        """
        Extrai e match keypoints e descritores entre duas imagens.

        Args:
            image0: Primeira imagem.
            image1: Segunda imagem.

        Returns:
            Dicionário com keypoints, descritores e matches.
        """
        image0 = input["image0"]
        image1 = input["image1"]
        lafs0, responses0, descriptors0 = self.local_feature(image0)
        lafs1, responses1, descriptors1 = self.local_feature(image1)
        distance, matches = self.descriptor_matcher(descriptors0[0], descriptors1[0])
        return {
            "keypoints0": lafs0[0, :, :, 2].data,#[N, 2])
            "keypoints1": lafs1[0, :, :, 2].data,#[N, 2])
            "lafs0": lafs0,#[1, N, 2, 3]
            "lafs1": lafs1,#[1, N, 2, 3]
            "descriptors0": descriptors0[0],#[N, 128])
            "descriptors1": descriptors1[0],#[N, 128])
            "matches": matches,#[M, 2])
        }

import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
class DelaunayGraph(IGlobalFeatureStructurer):
    @staticmethod
    def distancePoint(p1, p2):
        return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

    def buildConnection(self,tri):
        mapConnection = []
        for simplice in tri.simplices:
            mapConnection.extend([
                [simplice[0], simplice[1]],
                [simplice[1], simplice[2]],
                [simplice[2], simplice[0]]
            ])
        return mapConnection

    def buildMapGraph(self, mapConex,featuresByPoints):
        size = len(featuresByPoints)
        sample = np.matrix(np.ones((size, size)) * np.inf)
        for i in range(size):
            sample[i, i] = 0
        for conexao in mapConex:
            distance = self.distancePoint(featuresByPoints[conexao[0]], featuresByPoints[conexao[1]])
            sample[conexao[0], conexao[1]] = distance
        return sample

    @staticmethod
    def plot_delaunay(points, tri):
        plt.triplot(points[:, 0], points[:, 1], tri.simplices.copy(), color='orange')
        plt.plot(points[:, 0], points[:, 1], 'o')
        dist = 3
        for i, point in enumerate(points):
            plt.text(point[0] + dist, point[1] + dist, f' {i}', color='red', fontsize=12)
        plt.show()

    def __call__(self, points, featuresByPoints):
        # Assume-se que os pontos são extraídos de uma imagem e já processados
        tri = Delaunay(points)
        mapConnection = self.buildConnection(tri)
        graph = self.buildMapGraph(mapConnection,featuresByPoints)
        return graph,tri


from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall
class FloydWarshall(IGlobalMatcher):
    @staticmethod
    def floydWarshall(graph):
        graph = csr_matrix(graph)
        dist_matrix, _ = floyd_warshall(csgraph=graph, directed=False, return_predecessors=True)
        return dist_matrix

    @staticmethod
    def match_matrix(mat_a, mat_b, threshold):
        mat_dist = mat_b - mat_a
        for i in range(mat_dist.shape[0]):
            mat_dist[i, :i] = 0  # Zera a metade inferior da matriz para evitar duplicação
        mat_dist = mat_dist * mat_dist  # Eleva as diferenças ao quadrado
        points = 0
        for i in range(mat_dist.shape[0]):
            for j in range(i + 1, mat_dist.shape[0]):
                if mat_dist[i, j] < threshold:
                    points += 1
        return points

    def __call__(self,matrixAdj0,matrixAdj1, threshold=0.2):
        matAdjFull0 = self.floydWarshall(matrixAdj0)
        matAdjFull1 = self.floydWarshall(matrixAdj1)
        simGraph = self.match_matrix(matAdjFull0,matAdjFull1,threshold)
        return simGraph

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)
from kornia.feature import SIFTFeatureScaleSpace,SIFTFeature
my_sift_scale_matcher = MyLocalFeatureMatcher(
    SIFTFeatureScaleSpace(25,device=device), DescriptorMatcher('smnn', 0.8),#GFTTAffNetHardNet, KeyNetHardNet, SIFTFeature, SIFTFeatureScaleSpace
)

# image1 = torchvision.io.read_image("image1.jpg").to(device)
# image2 = torchvision.io.read_image("image2.jpg").to(device)
# image3 = torchvision.io.read_image("image2.jpg").to(device)
# pp= PreprocessPipeline()


# transform_original =torchvision.transforms.Compose([
#             torchvision.transforms.ToPILImage(),
#             torchvision.transforms.Resize((300, 300)),
#             torchvision.transforms.PILToTensor(),
#             torchvision.transforms.ConvertImageDtype(torch.float32),
#         ])

# with medir_tempo('Processamento concluído'):
#   image1_p = transform_original(image1)
#   image1_p = pp(image1_p)
#   print(image1_p[:,:1].shape,image1_p[0].shape,image1.shape)
#   input = {"image0": image1_p[:,:1], "image1":image1_p[:,:1]}
#   out = my_sift_scale_matcher(input)

# plot_image_with_keypoints(image1_p[0].cpu(),out['keypoints0'].detach().cpu())
# out.keys()


#avaliar com e sem gpu ok
#avaliar o pipeline


In [ ]:

from kornia.feature import SIFTFeatureScaleSpace
my_sift_scale_matcher = MyLocalFeatureMatcher(
    SIFTFeatureScaleSpace(10,device=device), DescriptorMatcher('smnn', 0.8),#GFTTAffNetHardNet, KeyNetHardNet, SIFTFeature, SIFTFeatureScaleSpace
)
pp= PreprocessPipeline()
# image1 = torchvision.io.read_image("image1.jpg").to(device)
#
# with medir_tempo('Processamento concluído'):
#   image1_p = pp(image1)
#   print(image1_p[:,:1].shape,image1_p[0].shape,image1.shape)
#   input = {"image0": image1_p[:,:1], "image1":image1_p[:,:1]}
#   out = my_sift_scale_matcher(input)

# plot_image_with_keypoints(image1_p[0].cpu(),out['keypoints0'].detach().cpu())
# out.keys()


#avaliar com e sem gpu
#avaliar o pipeline


In [ ]:
delaunayG = DelaunayGraph()  # Inicializa o construtor de grafo de Delaunay
# Compara as matrizes de distância dos dois grafos
floyd = FloydWarshall()

In [ ]:


class ImageComparisonPipeline:
    __slots__ = ['preprocessor', 'local_feature_matcher', 'global_structurer', 'global_matcher']

    def __init__(self, preprocessor: IPreprocessor = None, local_feature_matcher: MyLocalFeatureMatcher = None, global_structurer: IGlobalFeatureStructurer = None, global_matcher: IGlobalMatcher = None):
        self.preprocessor = preprocessor
        self.local_feature_matcher = local_feature_matcher
        self.global_structurer = global_structurer
        self.global_matcher = global_matcher

    def process_global(self,out,index):
        pts = out[f'keypoints{index}'][out['matches'][:, index]].detach().numpy()  # Pontos de keypoints correspondentes
        desc = out[f'descriptors{index}'][out['matches'][:, index]].detach().numpy()  # Descritores dos keypoints correspondentes
        matrixAdj, tri = self.global_structurer(pts, desc)
        return matrixAdj

    def run(self, image1: Any, image2: Any,log=None) -> float:
        print(image1.shape,image2.shape)
        if not all([self.preprocessor, self.local_feature_matcher, self.global_structurer, self.global_matcher]):
            raise ValueError("Pipeline components are not fully set.")
        image1_processed, image2_processed = map(self.preprocessor, (image1, image2))
        print(image1_processed.shape,image1_processed[:,:1].shape)
        input = {"image0": image1_processed[:,:1], "image1":image2_processed[:,:1]}
        print("local_feature_matching",image1_processed.shape,type(image1_processed))
        # Realiza o matching de características entre as duas imagens processadas
        out = self.local_feature_matcher(input)

        if log is not None and log in ('DEBUG','INFO'):
          print(out['keypoints1'].shape,type(out['keypoints1']),'\n',
                out['lafs1'].shape,type(out['lafs1']),'\n',
                out['descriptors1'].shape,type(out['descriptors1']),'\n',
                out['matches'].shape,)
        try:
          matrixAdj0 = self.process_global(out,0)
          matrixAdj1 = self.process_global(out,1)
          score = self.global_matcher(matrixAdj0, matrixAdj1, threshold=0.005)
        except:
          score = 0
        return score

pipeline = ImageComparisonPipeline(pp,my_sift_scale_matcher,delaunayG,floyd)
# try:
#     similarity_score = pipeline.run(image1, image1)
#     print(f"Similarity Score: {similarity_score}")
# except Exception as e:
#     print(f"Erro durante a execução do pipeline: {e}")

In [ ]:
class MyDrawMatcher:
    def __init__(self,draw_dict=None) -> None:
      if draw_dict is None:
        draw_dict = {
            "inlier_color": (0.2, 1, 0.2),
            "tentative_color": (1.0, 0.5, 1),
            "feature_color": (0.6, 0.5, 0),
            "vertical": False,
        }
      self.draw_dict = draw_dict

    def __call__(self,img1_preprocessed, img2_preprocessed, output) -> None:
            from kornia_moons.viz import draw_LAF_matches
            # Use kornia.tensor_to_image to ensure the images are in the correct format for drawing
            img1 = K.tensor_to_image(img1_preprocessed.squeeze())
            img2 = K.tensor_to_image(img2_preprocessed.squeeze())
            # Assumes draw_LAF_matches is accessible and compatible with provided arguments
            draw_LAF_matches(
                output['lafs0'].cpu(),
                output['lafs1'].cpu(),
                output['matches'].cpu(),
                img1,
                img2,
                None,  # Or the inliers if available
                self.draw_dict
            )

In [33]:
set_seed(42)


class ImageComparisonPipeline:
    __slots__ = ['preprocessor', 'local_feature', 'descriptor_matcher', 'global_structurer', 'global_matcher']

    def __init__(self, preprocessor=None, local_feature: LocalFeature = None, descriptor_matcher: DescriptorMatcher = None, global_structurer=None, global_matcher=None):
        self.preprocessor = preprocessor
        self.local_feature = local_feature
        self.descriptor_matcher = descriptor_matcher
        self.global_structurer = global_structurer
        self.global_matcher = global_matcher

    def process_global(self, out):
        matricesAdj = []
        for index in (0, 1):
            pts = out[f'keypoints{index}'][out['matches'][:, index]].cpu().detach().numpy()
            desc = out[f'descriptors{index}'][out['matches'][:, index]].cpu().detach().numpy()
            matrixAdj, _ = self.global_structurer(pts, desc)
            matricesAdj.append(matrixAdj)
        return matricesAdj

    def local_feature_matching(self, image0, image1):
        lafs0, responses0, descriptors0 = self.local_feature(image0)
        # print('local_feature_matching ',lafs0.shape,lafs0.device)
        lafs1, responses1, descriptors1 = self.local_feature(image1)
        # print('local_feature_matching ',lafs1.shape,lafs1.device)
        distance, matches = self.descriptor_matcher(descriptors0[0], descriptors1[0])
        return {
            "keypoints0": lafs0[0, :, :, 2].data,#[N, 2])
            "keypoints1": lafs1[0, :, :, 2].data,#[N, 2])
            "lafs0": lafs0,#[1, N, 2, 3]
            "lafs1": lafs1,#[1, N, 2, 3]
            "descriptors0": descriptors0[0],#[N, 128])
            "descriptors1": descriptors1[0],#[N, 128])
            "matches": matches,#[M, 2])
        }

    def run(self, inspection_images: torch.Tensor, reference_images: torch.Tensor,threshold=0.1, log=None,device=torch.device('cpu')) -> Dict[Tuple[int, int], float]:
        if not all([self.preprocessor, self.local_feature, self.descriptor_matcher, self.global_structurer, self.global_matcher]):
            raise ValueError("Pipeline components are not fully set.")
        n,m = inspection_images.shape[0],reference_images.shape[0]
        scores = np.zeros((n, m))
        count_match = np.zeros((n, m))
        myDraw =MyDrawMatcher()
        for i_index, i_image in enumerate(inspection_images):
            lafs0, responses0, descriptors0 = self.local_feature(i_image[:1][None])
            for r_index, r_image in enumerate(reference_images):

                lafs1, responses1, descriptors1 = self.local_feature(r_image[:1][None])
                distance, matches = self.descriptor_matcher(descriptors0[0], descriptors1[0])
                out = {
                "keypoints0": lafs0[0, :, :, 2].data,#[N, 2])
                "keypoints1": lafs1[0, :, :, 2].data,#[N, 2])
                "lafs0": lafs0,#[1, N, 2, 3]
                "lafs1": lafs1,#[1, N, 2, 3]
                "descriptors0": descriptors0[0],#[N, 128])
                "descriptors1": descriptors1[0],#[N, 128])
                "matches": matches,#[M, 2])
                }

                # out = self.local_feature_matching(i_image[:1][None], r_image[:1][None])

                try:
                    matricesAdj = self.process_global(out)
                    score = self.global_matcher(*matricesAdj, threshold=threshold)
                except Exception as e:
                    score = 0
                if log is not None and log in ('DEBUG') and i_index == r_index:
                    print(i_index,r_index ,out['keypoints0'].shape,out['descriptors0'].shape,out['matches'].shape,)
                    myDraw(i_image.cpu(), r_image.cpu(),out)

                count_match[i_index, r_index] = matches.shape[0]
                scores[i_index, r_index] = score

        if log is not None and log in ('INFO'):
            print_table(count_match)
            print(count_correct_matches(count_match))
            print_table(scores)
            print(count_correct_matches(scores))

        return count_match,scores


import torch
from torchvision import datasets, transforms
import kornia.augmentation as KA
from torch.utils.data import DataLoader, Dataset
from kornia.feature import GFTTAffNetHardNet, KeyNetHardNet, SIFTFeature, SIFTFeatureScaleSpace
#GFTTAffNetHardNet, KeyNetHardNet, SIFTFeature, SIFTFeatureScaleSpace/ SIFTFeature(pior desempenho)

pipeline = ImageComparisonPipeline(pp, GFTTAffNetHardNet(50,device=device), DescriptorMatcher('smnn', 1.5), delaunayG, floyd)


transform_original = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor()
])

transform_inspect = KA.AugmentationSequential(
    # KA.RandomHorizontalFlip(p=0.5),
    # KA.RandomVerticalFlip(p=0.5),
    KA. RandomMedianBlur((3, 3), p = 1),
    KA.RandomPerspective(0.3, p=0.75),
    KA.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.25, p=0.75),#brightness=0.5, contrast=0.5, saturation=0.5, hue=0.15, p=0.75
    KA.RandomAffine(degrees=90, translate=(0.10, 0.10), scale=(0.95, 1.05), p=0.75),
    same_on_batch=True,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device',device)
flowers102Dataset = datasets.Flowers102(root='./data', split='train', download=True,transform=transform_original)
flowers_loader = DataLoader(flowers102Dataset, batch_size=20, shuffle=True,)
# for batch_idx, (original_batch, target) in enumerate(flowers_loader):
#     original_batch = original_batch.to(device)
#     reference_images = original_batch
#     inspection_images =  transform_inspect(original_batch)
#     with medir_tempo():
#       matchs,scores = pipeline.run(inspection_images, original_batch,device=device,log='INFO')
#     break


device cuda


In [34]:
import torch
from itertools import product

def grid_search_pipeline(device, dataset_loader, param_grid):
    all_results = []
    transform_inspect = KA.AugmentationSequential(
        # KA.RandomHorizontalFlip(p=0.5),
        # KA.RandomVerticalFlip(p=0.5),
        KA. RandomMedianBlur((3, 3), p = 1),
        KA.RandomPerspective(0.3, p=0.75),
        KA.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.25, p=0.75),#brightness=0.5, contrast=0.5, saturation=0.5, hue=0.15, p=0.75
        KA.RandomAffine(degrees=90, translate=(0.10, 0.10), scale=(0.95, 1.05), p=0.75),
        same_on_batch=True,
    )
    for num_features, feature_local_class, distance, threshold in product(*param_grid.values()):
        set_seed(42)
        # Configuração específica do extrator de features com a classe e número de features
        feature_extractor = feature_local_class(num_features, device=device)
        
        # Configurar a pipeline com os parâmetros atuais
        pipeline = ImageComparisonPipeline(
            pp, 
            feature_extractor, 
            DescriptorMatcher('smnn', distance), 
            delaunayG, 
            floyd,
        )
        
        # Executar a pipeline e registrar resultados para o batch atual
        for batch_idx, (original_batch, target) in enumerate(dataset_loader):
            original_batch = original_batch.to(device)
            inspection_images = transform_inspect(original_batch)
            with medir_tempo():
                matches, scores = pipeline.run(inspection_images, original_batch,threshold=threshold, device=device, log=None)
            all_results.append({
                'params': {
                    'num_features': num_features,
                    'feature_local_class': feature_local_class.__name__,
                    'distance': distance,
                    'threshold': threshold
                },
                'matches': count_correct_matches(matches),
                'scores': count_correct_matches(scores),
            })
            break # Remover este break para executar em todo o dataset
    
    return all_results

# Grid de parâmetros
param_grid = {
    'num_features': [10, 15],   
    'feature_local': [GFTTAffNetHardNet, KeyNetHardNet, SIFTFeature, SIFTFeatureScaleSpace],
    'distance': [1.5, 1.75],
    'threshold': [0.005,0.5]
}

# Supondo que 'flowers_loader' seja o DataLoader do seu dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
all_results = grid_search_pipeline(device, flowers_loader, param_grid)
for result in all_results:
    print("Parâmetros:", result['params'])
    print("Scores:", result['scores'])


Tempo de execução: 17.590347621000546 segundos
Tempo de execução: 17.59572443900106 segundos
Tempo de execução: 17.281087627001398 segundos
Tempo de execução: 17.864927881999392 segundos
Tempo de execução: 17.71316234599908 segundos
Tempo de execução: 17.509508766999716 segundos


In [30]:
from prettytable import PrettyTable


# Criando uma tabela com prettytable
table = PrettyTable()

# Adicionando cabeçalhos à tabela
table.field_names = ["Num Features", "Feature Local Class", "Distance", "Threshold", "Matches", "Scores"]

# Adicionando as linhas com os resultados
for result in all_results:
    params = result['params']
    matches = result['matches']
    scores = result['scores']
    table.add_row([params['num_features'], params['feature_local_class'], params['distance'], params['threshold'], matches, scores])

# Imprimindo a tabela
print(table)
# Gerar a string CSV da tabela
csv_string = table.get_csv_string()

# Salvar a string CSV em um arquivo
with open('resultados_grid_search.csv', 'w') as csv_file:
    csv_file.write(csv_string)

+--------------+-----------------------+----------+-----------+----------+----------+
| Num Features |  Feature Local Class  | Distance | Threshold | Matches  |  Scores  |
+--------------+-----------------------+----------+-----------+----------+----------+
|      30      |   GFTTAffNetHardNet   |   0.95   |   0.005   | (25, 30) | (27, 30) |
|      30      |   GFTTAffNetHardNet   |   0.95   |    0.05   | (25, 30) | (26, 30) |
|      30      |   GFTTAffNetHardNet   |   0.95   |    0.5    | (25, 30) | (26, 30) |
|      30      |   GFTTAffNetHardNet   |   1.5    |   0.005   | (18, 30) | (17, 30) |
|      30      |   GFTTAffNetHardNet   |   1.5    |    0.05   | (18, 30) | (16, 30) |
|      30      |   GFTTAffNetHardNet   |   1.5    |    0.5    | (18, 30) | (18, 30) |
|      30      |   GFTTAffNetHardNet   |   1.75   |   0.005   | (18, 30) | (17, 30) |
|      30      |   GFTTAffNetHardNet   |   1.75   |    0.05   | (18, 30) | (16, 30) |
|      30      |   GFTTAffNetHardNet   |   1.75   |   